<a href="https://colab.research.google.com/github/owosade/Explainable-AI---Final/blob/main/Lime.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

# Mount Google Drive to '/content/gdrive'
drive.mount('/content/gdrive')

!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()
!pip install lime
import lime
from lime import lime_image
from fastai.vision.all import *


Mounted at /content/gdrive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.8/719.8 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 49.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 6.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lime: filename=lime-0.2.0.1-py3-none-any.whl size=283835 sha256=33f94c8e84b00d2c43650f9506558310f7f3281ad9534060895f5294611be475
  Stored in directory: /root/.cache/pip/wheels/fd/a2/af/9ac0a1a85a27f314a06b39e1f492bee1547d52549a4606ed89
Successfully built lime


In [ ]:
path = Path('/content/gdrive/MyDrive/Dataset')
#dls = ImageDataLoaders.from_folder(path/'Kidney_stones', train='train', valid='valid')
train_fnames = get_image_files(path/"Train")
augs = [RandomResizedCropGPU(size=224, min_scale=0.75), Rotate(), Zoom()]


dblock = DataBlock(blocks=(ImageBlock(cls=PILImage), CategoryBlock),
                   splitter=RandomSplitter(valid_pct=0.2, seed=23),
                   get_y=parent_label,
                   item_tfms=Resize(512, method="squish"),
                   batch_tfms=augs,
                   )

dls = dblock.dataloaders(train_fnames)
# Step 2: Create a fastai Learner
learn = cnn_learner(dls, resnet34, metrics=accuracy)
learn = learn.load('/content/gdrive/MyDrive/Dataset/Final_project4')

/usr/local/lib/python3.10/dist-packages/fastai/vision/learner.py:301: UserWarning: `cnn_learner` has been renamed to `vision_learner` -- please update your code
  warn("`cnn_learner` has been renamed to `vision_learner` -- please update your code")
Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth
100%|██████████| 83.3M/83.3M [00:01<00:00, 79.1MB/s]


In [ ]:
import cv2
from lime import lime_image
from skimage.segmentation import mark_boundaries
from PIL import Image
# Step 3: Prepare your data
# Assuming 'image_path' is the path to an image in your dataset
#image_path = cv2.imread('/content/gdrive/MyDrive/Dataset/Test/Kidney_stone/1.3.46.670589.33.1.63706830475347975400001.4676991400730475635.png')
image_path =('/content/gdrive/MyDrive/Dataset/Test/Kidney_stone/1.3.46.670589.33.1.63706830475347975400001.4676991400730475635.png')
#img_gray = cv2.cvtColor(image_path, cv2.COLOR_BGR2GRAY)
img = PILImage.create(image_path)

img = np.array(img)
img = img.astype(np.float32)/ 255.0
# Step 4: Create a LIME explainer
explainer = lime_image.LimeImageExplainer()

# Use the Fastai Learner to make a prediction


# Extract the predicted probabilities from the Category object


# Step 5: Generate explanations
# The predict function should take a PIL Image and return model predictions
def predict_function(img):
    #return learn.predict(PILImage.create(img))[2].numpy()
    pred = learn.predict(PILImage.create(img))[0]
    return pred.probs.numpy()

# Explain the predictions for the given image
explanation = explainer.explain_instance(img, predict_function, top_labels=1, hide_color=0, num_samples=1000)

# Step 6: Visualize the explanations
from lime.wrappers.scikit_image import SegmentationAlgorithm

# Use a segmentation algorithm for better visualization (optional)
segmenter = SegmentationAlgorithm('quickshift', kernel_size=1, max_dist=200, ratio=0.2)

# Visualize the explanation
explanation.show_in_notebook(image_path=image_path, segments=segmenter)

  0%|          | 0/1000 [00:00<?, ?it/s]

TypeError: Cannot handle this data type: (1, 1, 1052, 3), <f4